# Basic Imports

In [0]:
import pandas as pd
import numpy as np
import os, shutil

from imageio import imread
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from itertools import chain
import itertools
import logging
import warnings

from torch.nn import Sigmoid
from torch import Tensor, LongTensor
from torch.autograd import Variable

import torchvision.transforms as transforms

import torch
import torchvision
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from PIL import Image
from torch import optim
import matplotlib.pyplot as plt
import math
import torchvision.utils as vutils

%matplotlib inline
%load_ext tensorboard
warnings.filterwarnings('ignore')

import time, datetime

def delete_logs():
  folder = "./logs"
  for filename in os.listdir(folder):
      file_path = os.path.join(folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              os.unlink(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))

#delete_logs()
def get_tensorboard_logger():
  logs_base_dir = "./logs"
  logs_dir = logs_base_dir + "/run_" + str(time.mktime(datetime.datetime.now().timetuple()))
  os.makedirs(logs_dir, exist_ok=True)
  from torch.utils.tensorboard import SummaryWriter
  %tensorboard --logdir {logs_base_dir} --port=9002
  logger = SummaryWriter(logs_dir)
  return logger

In [2]:
!nvidia-smi

Sun Dec 15 02:33:42 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 441.12       Driver Version: 441.12       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P1000       WDDM  | 00000000:01:00.0  On |                  N/A |
| 38%   46C    P8    N/A /  N/A |    448MiB /  4096MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Datasets

## MNIST Dataset

In [0]:
# from torchvision.datasets import MNIST
# from torchvision import datasets
# from torchvision import transforms

# batch_size = 50
# train_loader = torch.utils.data.DataLoader(
#         datasets.MNIST('../data', train=True, download=True,
#                        transform=transforms.Compose([
#                            transforms.ToTensor(),
#                            transforms.Normalize((0.1307,), (0.3081,))
#                        ])),
#         batch_size=batch_size, shuffle=True )

# test_loader = torch.utils.data.DataLoader(
#         datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                            transforms.ToTensor(),
#                            transforms.Normalize((0.1307,), (0.3081,))
#                        ])),
#         batch_size=batch_size, shuffle=True)

## Cluttered MNIST
https://github.com/kvn219/cluttered-mnist/blob/master/spatial-transformer-network/Clutter_MNIST_Example.ipynb

In [0]:
class CTrainDataset():
  def __init__(self, input_path):    
    mnist_cluttered = np.load(input_path)
    self.X_train = mnist_cluttered['X_train']
    self.y_train = mnist_cluttered['y_train']
    self.length = len(self.X_train)
  
  def __len__(self):
    return self.length 
  
  def __getitem__(self, index):
    return self.X_train[index].reshape((1, 40, 40)), self.y_train[index][0]

def custom_collate(batch):
  return default_collate(batch)

class CTestDataset():
  def __init__(self, input_path):    
    mnist_cluttered = np.load(input_path)
    self.X_test = mnist_cluttered['X_test']
    self.y_test = mnist_cluttered['y_test']
    self.length = len(self.X_test)
  
  def __len__(self):
    return self.length 
  
  def __getitem__(self, index):
    return self.X_test[index].reshape((1, 40, 40)), self.y_test[index][0]

def custom_collate(batch):
  return default_collate(batch)

In [0]:
# batch_size = 1000
# train_dataset = CTrainDataset('FairRaceDetection/data/mnist_sequence1_sample_5distortions5x5.npz')
# test_dataset = CTestDataset('FairRaceDetection/data/mnist_sequence1_sample_5distortions5x5.npz')
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False )
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=36, shuffle=False )

# idx = 6
# img, y = train_dataset[idx]
# img = img[0]
# print(np.max(img))
# print(y)
# plt.imshow(img)
# plt.show()

## Temp Dataset

In [0]:
class TempDataSet():
  def __len__(self):
    return 2000
  
  def __getitem__(self, index):
    image = torch.randn((3, 128, 128)).numpy()
    labels = np.array([0.0, 100.0])
    return image.astype(float), labels.astype(float)    

## Cancer Dataset

In [0]:
class CancerDataset(torch.utils.data.Dataset):
  def __init__(self, dataFolder, dataList):
    self.data = dataList
    self.dataFolder = dataFolder
    
  def __len__(self):
    return 128 #len(self.data)
    
  def __getitem__(self, index):
    filename = self.data.iloc[index].filename
    if 'FullRes' in filename :
        #Removes the starting "FullRes" from the filename
        filename = filename[7:]
    data = imread(os.path.join(self.dataFolder, filename))
    data = Image.fromarray(data)
    data = data.resize((256,256))
    data = np.array(data)
    data = data/256.0 
    data = torchvision.transforms.ToTensor()(data)
    data = data.float()
    label = self.data.iloc[index].label
    bmi = self.data.iloc[index].BMI
    return data, np.array([label, bmi]).astype(float)

#Resnet Model Classes

## Attention Blocks

In [0]:
def get_norm(norm_type, num_features, num_groups=32, eps=1e-5):
  if norm_type == 'BatchNorm':
    return nn.BatchNorm2d(num_features, eps=eps)
  elif norm_type == "GroupNorm":
    return nn.GroupNorm(num_groups, num_features, eps=eps)
  elif norm_type == "InstanceNorm":
    return nn.InstanceNorm2d(num_features, eps=eps, affine=True, track_running_stats=True)
  else:
    raise Exception('Unknown Norm Function : {}'.format(norm_type))


In [0]:
import torch
from torch import nn
from torch.nn import functional as F

class Conv2dNormRelu(nn.Module):
  def __init__(self, in_ch, out_ch, kernel_size=3, stride=1, padding=0, bias=True, norm_type='Unknown'):
    super(Conv2dNormRelu, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias=bias),
        get_norm(norm_type, out_ch),
        nn.ReLU(inplace=True))
  def forward(self, x):
    return self.conv(x)


class CAModule(nn.Module):
  def __init__(self, num_channels, reduc_ratio=2):
    super(CAModule, self).__init__()
    self.num_channels = num_channels
    self.reduc_ratio = reduc_ratio
    self.fc1 = nn.Linear(num_channels, num_channels // reduc_ratio, bias=True)
    self.fc2 = nn.Linear(num_channels // reduc_ratio, num_channels, bias=True)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
  def forward(self, feat_map):
    gap_out = feat_map.view(feat_map.size()[0], self.num_channels, -1).mean(dim=2)
    fc1_out = self.relu(self.fc1(gap_out))
    fc2_out = self.sigmoid(self.fc2(fc1_out))
    fc2_out = fc2_out.view(fc2_out.size()[0], fc2_out.size()[1], 1, 1)
    feat_map = torch.mul(feat_map, fc2_out)
    return feat_map, [fc2_out]

class SAModule(nn.Module):
  def __init__(self, num_channels):
    super(SAModule, self).__init__()
    self.num_channels = num_channels
    self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=num_channels // 8, kernel_size=1)
    self.conv2 = nn.Conv2d(in_channels=num_channels, out_channels=num_channels // 8, kernel_size=1)
    self.conv3 = nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=1)
    self.gamma = nn.Parameter(torch.zeros(1))
  
  def forward(self, feat_map):
    batch_size, num_channels, height, width = feat_map.size()
    conv1_proj = self.conv1(feat_map).view(batch_size, -1, width * height).permute(0, 2, 1)
    conv2_proj = self.conv2(feat_map).view(batch_size, -1, width * height)
    relation_map = torch.bmm(conv1_proj, conv2_proj)
    attention = F.softmax(relation_map, dim=-1)
    conv3_proj = self.conv3(feat_map).view(batch_size, -1, width * height)
    feat_refine = torch.bmm(conv3_proj, attention.permute(0, 2, 1))
    feat_refine = feat_refine.view(batch_size, num_channels, height, width)
    feat_map = self.gamma * feat_refine + feat_map
    print(feat_refine.shape, attention.shape)
    return feat_map, [self.gamma * feat_refine, feat_refine]


class FPAModule(nn.Module):
  def __init__(self, num_channels, norm_type):
    super(FPAModule, self).__init__()
    self.gap_branch = nn.Sequential(nn.AdaptiveAvgPool2d(1), Conv2dNormRelu(num_channels, num_channels, kernel_size=1, norm_type=norm_type))
    self.mid_branch = Conv2dNormRelu(num_channels, num_channels, kernel_size=1, norm_type=norm_type)
    self.downsample1 = Conv2dNormRelu(num_channels, 1, kernel_size=3, stride=2, padding=1, norm_type=norm_type)
    self.downsample2 = Conv2dNormRelu(1, 1, kernel_size=3, stride=2, padding=1, norm_type=norm_type)
    # self.downsample3 = Conv2dNormRelu(1, 1, kernel_size=3, stride=2, padding=1, norm_type=norm_type)
    self.scale1 = Conv2dNormRelu(1, 1, kernel_size=3, padding=1, norm_type=norm_type)
    self.scale2 = Conv2dNormRelu(1, 1, kernel_size=3, padding=1, norm_type=norm_type)
    # self.scale3 = Conv2dNormRelu(1, 1, kernel_size=3, padding=1, norm_type=norm_type)
  
  def forward(self, feat_map):
    height, width = feat_map.size(2), feat_map.size(3)
    gap_branch = self.gap_branch(feat_map)
    gap_branch = nn.Upsample(size=(height, width), mode='bilinear', align_corners=False)(gap_branch)
    mid_branch = self.mid_branch(feat_map)
    scale1 = self.downsample1(feat_map)
    scale2 = self.downsample2(scale1)
    # scale3 = self.downsample3(scale2)
    # scale3 = self.scale3(scale3)
    # scale3 = nn.Upsample(size=(int(math.ceil(height*1.0 / 4.0)), int(math.ceil(width*1.0 / 4.0))), mode='bilinear', align_corners=False)(scale3)
    # scale2 = self.scale2(scale2) + scale3
    scale2 = self.scale2(scale2)
    scale2 = nn.Upsample(size=(int(math.ceil(height*1.0 / 2.0)), int(math.ceil(width*1.0 / 2.0))), mode='bilinear', align_corners=False)(scale2)
    scale1 = self.scale1(scale1) + scale2
    scale1 = nn.Upsample(size=(height, width), mode='bilinear', align_corners=False)(scale1)
    feat_map = torch.mul(scale1, mid_branch) + gap_branch
    return feat_map, []

class AttMap(nn.Module):
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(AttMap, self).__init__()
        norm_layer = nn.BatchNorm2d
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        # self.conv3 = conv1x1(planes,1)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        # out = self.conv3(out)
        out = torch.sigmoid(out)
        return out, []

## ResNet Wrapper

In [10]:
from torchvision.models.utils import load_state_dict_from_url
from torchvision.models.resnet import BasicBlock, _resnet, conv3x3, conv1x1
import torch.nn as nn 

global_batchid = -1
global_epoch = -1
global_id = 0
global_logger = get_tensorboard_logger()

class AdvancedBasicBlock(BasicBlock):
  def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
    super(AdvancedBasicBlock, self).__init__(inplanes, planes, stride=stride, downsample=downsample, groups=groups,
                 base_width=base_width, dilation=dilation, norm_layer=norm_layer)
    
    print(self.bn2.num_features)
    global global_id
    self.id = global_id
    global_id += 1
    self.attention_layer = None
  
  def forward(self, x):
    identity = x
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    
    out = self.conv2(out)
    out = self.bn2(out)
    attention_map, plot_data = self.attention_layer(x)
    # plot_data.append(out)
    # plot_data.append(out * attention_map)
    out = out * attention_map + out
    # plot_data.append(out)
    global global_batchid, global_epoch, global_logger

    local_batch_id = global_batchid
    local_epoch = global_epoch

    if not self.training and local_batch_id < 1:
      for index, value in enumerate(plot_data):
        for channel_id in range(value.shape[1]):
          if(channel_id > 10):
            continue
          key = "_ListId_" + str(index) + "_Layer_" + str(self.id) + "_BatchId_" + str(local_batch_id) + "_ChannelId_" + str(channel_id)
          sample = value[:, channel_id, :, :].unsqueeze(1)
          # sample = sample.repeat(1, 3, 1, 1)
          global_logger.add_images(key, vutils.make_grid(sample.cpu(), padding=2).unsqueeze(0),  local_epoch)
          
    
    if self.downsample is not None:
      identity = self.downsample(x)
    
    out += identity
    out = self.relu(out)
    return out

class AdvancedBasicBlock_CAM(AdvancedBasicBlock):
  def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1, 
               base_width=64, dilation=1, norm_layer=None):
    super(AdvancedBasicBlock_CAM, self).__init__(inplanes, planes, stride=stride, downsample=downsample, groups=groups,
                 base_width=base_width, dilation=dilation, norm_layer=norm_layer)
    self.attention_layer = CAModule(self.bn2.num_features)

class AdvancedBasicBlock_SAM(AdvancedBasicBlock):
  def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1, 
               base_width=64, dilation=1, norm_layer=None):
    super(AdvancedBasicBlock_SAM, self).__init__(inplanes, planes, stride=stride, downsample=downsample, groups=groups,
                 base_width=base_width, dilation=dilation, norm_layer=norm_layer)
    self.attention_layer = SAModule(self.bn2.num_features)

class AdvancedBasicBlock_FPA(AdvancedBasicBlock):
  def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1, 
               base_width=64, dilation=1, norm_layer=None):
    super(AdvancedBasicBlock_FPA, self).__init__(inplanes, planes, stride=stride, downsample=downsample, groups=groups,
                 base_width=base_width, dilation=dilation, norm_layer=norm_layer)
    self.attention_layer = FPAModule(self.bn2.num_features, 'BatchNorm')

class AdvancedBasicBlock_BASIC(AdvancedBasicBlock):
  def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1, 
               base_width=64, dilation=1, norm_layer=None):
    super(AdvancedBasicBlock_BASIC, self).__init__(inplanes, planes, stride=stride, downsample=downsample, groups=groups,
                 base_width=base_width, dilation=dilation, norm_layer=norm_layer)
    self.attention_layer = AttMap(inplanes, planes, stride=stride, downsample=downsample, groups=groups,
                 base_width=base_width, dilation=dilation, norm_layer=norm_layer)

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 13488.

## OurResnet

In [0]:
from torchvision.models.resnet import ResNet

class OurResnet(ResNet):
  def __init__(self, block, layers, out_channels, num_classes=1000, zero_init_residual=False, groups=1, width_per_group=64, replace_stride_with_dilation=None, norm_layer=None):
    super(OurResnet, self).__init__(block, layers, num_classes=num_classes, zero_init_residual=zero_init_residual, groups=groups, width_per_group=width_per_group, 
                                    replace_stride_with_dilation=replace_stride_with_dilation, norm_layer=norm_layer)
    
    self.conv_final = nn.Conv2d(self.fc.in_features, out_channels, 2)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.conv_final(x)
    return x
   
def newResnet34(blk, pretrained=False, progress=True, out_channels=1, **kwargs):
  model = OurResnet(blk, [2,2,2,2], out_channels, **kwargs)
  return model

def oldResnet34(blk, pretrained=False, progress=True, out_channels=1, **kwargs):
  return _resnet('resnet34', blk, [3, 4, 6, 3], pretrained, progress, **kwargs)

# MNIST Network

## MNIST Network Wrapper

In [0]:
class Network():
  def __init__(self, bs=50, lr=0.001, epochs=10, model=None, mode='default'):
    self.lr = lr
    self.epochs = epochs
    self.batch_size = bs
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.kwargs = {'num_workers': 0, 'pin_memory': True} 
    if model is not None:
      self.model = model.to(self.device)
    else:
      if mode == 'default':
        resnet = torchvision.models.resnet34(pretrained=True)
      elif mode == 'sam':
        resnet = oldResnet34(AdvancedBasicBlock_SAM, pretrained=False)
        k = load_state_dict_from_url('https://download.pytorch.org/models/resnet34-333f7ec4.pth')
        resnet.load_state_dict(k, strict=False)
      elif mode == 'cam':
        resnet = oldResnet34(AdvancedBasicBlock_CAM, pretrained=False)
        k = load_state_dict_from_url('https://download.pytorch.org/models/resnet34-333f7ec4.pth')
        resnet.load_state_dict(k, strict=False)
      elif mode == 'fpa':
        resnet = oldResnet34(AdvancedBasicBlock_FPA, pretrained=False)
        k = load_state_dict_from_url('https://download.pytorch.org/models/resnet34-333f7ec4.pth')
        resnet.load_state_dict(k, strict=False)
      elif mode == 'basic':
        resnet = oldResnet34(AdvancedBasicBlock_BASIC, pretrained=False)
        k = load_state_dict_from_url('https://download.pytorch.org/models/resnet34-333f7ec4.pth')
        resnet.load_state_dict(k, strict=False)

      resnet.conv1 = nn.Conv2d(1, 64, 7, stride=2, padding=3, bias=False)
      resnet.fc = nn.Linear(512, 10)
      self.model = resnet.to(self.device)
      model_parameters = filter(lambda p: p.requires_grad, self.model.parameters())
      params = sum([np.prod(p.size()) for p in model_parameters])
      print("Number of Parameters: ", params)
    self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
    self.criterion = nn.CrossEntropyLoss()
    global global_logger
    self.logger = global_logger

  def train(self, train_loader, test_loader):
    self.model.train()
    for epoch in range(self.epochs):
      self.model.train()
      for batch_idx, (data, target) in enumerate(train_loader):
          data, target = data.to(self.device), target.to(self.device).long()
          self.optimizer.zero_grad()
          output = self.model(data)
          loss = self.criterion(output, target)
          loss.backward()
          self.optimizer.step()
          self.logger.add_scalar('Train_Loss', loss.item(), epoch*len(train_loader) + batch_idx)
          if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
      self.test(epoch, test_loader)  

  def test(self, epoch, test_loader):
    self.model.eval()
    test_loss = 0
    correct = 0
    global global_epoch, global_batchid
    global_epoch = epoch 
    with torch.no_grad():
        for batch_id, tup in enumerate(test_loader):
            global_batchid = batch_id
            data, target = tup
            data, target = data.to(self.device), target.to(self.device).long()
            # plt.imshow(data[0].transpose(0,1).transpose(1,2).squeeze())
            # plt.show()
            output = self.model(data)
            test_loss += self.criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            if(batch_id == 0):
              global_logger.add_images("Input Image", vutils.make_grid(data.cpu(), padding=2).unsqueeze(0),  epoch)
    test_loss /= len(test_loader)
    self.logger.add_scalar('Test_Loss', test_loss, epoch)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


## MNIST Train Network

In [0]:
# batch_size = 32
# resnet = Network(bs=batch_size, epochs=50, lr=0.001, mode='basic')
# resnet.train(train_loader=train_loader, test_loader=test_loader)


# Adversery Architecture



## Adversery Modules



In [0]:
class BaseNetwork(nn.Module):
  def __init__(self, mode):
    super(BaseNetwork, self).__init__()
    if mode == 'default':
      resnet = newResnet34(BasicBlock, out_channels=10)
    elif mode == 'sam':
      resnet = newResnet34(AdvancedBasicBlock_SAM, out_channels=10)
    elif mode == 'cam':
      resnet = newResnet34(AdvancedBasicBlock_CAM, out_channels=10)
    elif mode == 'fpa':
      resnet = newResnet34(AdvancedBasicBlock_FPA, out_channels=10)
    elif mode == 'basic':
      resnet = newResnet34(AdvancedBasicBlock_BASIC, out_channels=10)
    
    # k = load_state_dict_from_url('https://download.pytorch.org/models/resnet34-333f7ec4.pth')
    k = load_state_dict_from_url('https://download.pytorch.org/models/resnet18-5c106cde.pth')
    resnet.load_state_dict(k, strict=False)
    resnet.conv1 = nn.Conv2d(1, 64, 7, stride=2, padding=3, bias=False)
    self.model = resnet
    model_parameters = filter(lambda p: p.requires_grad, self.model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print("Number of Parameters: ", params)

  def forward(self, x):
      return self.model(x)

class RaceDetector(nn.Module):
  def __init__(self, in_shape):
    super(RaceDetector, self).__init__()
    self.l1 = nn.Linear(in_shape, 100)
    self.l2 = nn.Linear(100, 1)
  
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = self.l1(x)
    x = F.relu(x)
    x = self.l2(x)
    x = F.sigmoid(x)
    return x

class Adversary(nn.Module):
  def __init__(self, in_shape):
    super(Adversary, self).__init__()
    self.l1 = nn.Linear(in_shape, 100)
    self.l2 = nn.Linear(100, 1)
  
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = self.l1(x)
    x = F.relu(x)
    x = self.l2(x)
    return x

## Adversery Network Wrapper

In [0]:
class AdvNetworkWrapper():
  def __init__(self, basenetwork, racedetector, adversary, bs=50, lr=0.001, rd_epochs = 1, ad_epochs= 1, epochs=10, alpha = 0.8, pretrain_ad_epochs=5, pretrain_rd_epochs=5):
    self.lr = lr
    self.epochs = epochs
    self.batch_size = bs
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.kwargs = {'num_workers': 0, 'pin_memory': True} 
    self.basenetwork = basenetwork.to(self.device)
    self.racedetector = racedetector.to(self.device)
    self.adversary = adversary.to(self.device)
    self.rd_optimizer = optim.Adam(list(self.basenetwork.parameters()) + list(self.racedetector.parameters()), lr=self.lr)
    self.ad_optimizer = optim.Adam(self.adversary.parameters(), lr=self.lr)
    self.rd_criterion = nn.BCELoss()
    self.ad_criterion = nn.MSELoss()
    self.rd_epochs = rd_epochs
    self.ad_epochs = ad_epochs
    self.pretrain_ad_epochs = pretrain_ad_epochs
    self.pretrain_rd_epochs = pretrain_rd_epochs
    self.epochs = epochs
    self.alpha = alpha
    global global_logger
    self.logger = global_logger
  
  def train(self):
    self.basenetwork.train()
    self.adversary.train()
    self.racedetector.train()

  def eval(self):
    self.basenetwork.eval()
    self.adversary.eval()
    self.racedetector.eval()
  
  def pretrain_rd(self, train_loader):
    for epoch in range(self.pretrain_rd_epochs):
      self.train()
      for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(self.device).type(torch.float), target.to(self.device).type(torch.float)
        base_output = self.basenetwork(data)
        rd_output = self.racedetector(base_output)
        self.rd_optimizer.zero_grad()        
        rd_loss = self.rd_criterion(rd_output, target[:,0].unsqueeze(1))
        loss = rd_loss
        loss.backward()
        self.rd_optimizer.step()
        # Bookkeeping
        self.logger.add_scalar('PRE_TRAIN_RDNet_Loss', loss.item(), epoch*len(train_loader) + batch_idx)
        if batch_idx % 200 == 0:
          print('Pre Train Epoch: {} [{}/{} ({:.0f}%)]\tRD Total Loss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
        del base_output, rd_output, data, target
  
  def pretrain_ad(self, train_loader):
    for epoch in range(self.pretrain_ad_epochs):
      self.train()
      for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(self.device).type(torch.float), target.to(self.device).type(torch.float)
        base_output = self.basenetwork(data)
        adv_output = self.adversary(base_output.detach())
        self.ad_optimizer.zero_grad()
        loss = self.ad_criterion(adv_output, target[:,1].unsqueeze(1))
        loss.backward()
        self.ad_optimizer.step()

        # Bookkeeping
        self.logger.add_scalar('PRE_TRAIN_AdvNet_AdvLoss', loss.item(), epoch*len(train_loader) + batch_idx)
        if batch_idx % 200 == 0:
          print('Pre Train Epoch: {} [{}/{} ({:.0f}%)]\tAdv Loss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
        del base_output, adv_output, data, target

  def train_rd(self, train_loader, network_epoch):
    for epoch in range(self.rd_epochs):
      self.train()
      for batch_idx, (data, target) in enumerate(train_loader):
        self.train_adv(train_loader, network_epoch*self.rd_epochs*len(train_loader) + epoch*len(train_loader) + batch_idx)
        data, target = data.to(self.device).type(torch.float), target.to(self.device).type(torch.float)
        base_output = self.basenetwork(data)
        rd_output = self.racedetector(base_output)
        adv_output = self.adversary(base_output)
        
        self.rd_optimizer.zero_grad()        
        rd_loss = self.rd_criterion(rd_output, target[:,0].unsqueeze(1))
        adv_loss = self.ad_criterion(adv_output, target[:,1].unsqueeze(1))
        loss = rd_loss - self.alpha * adv_loss
        loss.backward()
        self.rd_optimizer.step()

        # Bookkeeping
        self.logger.add_scalar('RDNet_TotalLoss', loss.item(), network_epoch*self.rd_epochs*len(train_loader) + epoch*len(train_loader) + batch_idx)
        self.logger.add_scalar('RDNet_AdvLoss', adv_loss.item(), network_epoch*self.rd_epochs*len(train_loader) + epoch*len(train_loader) + batch_idx)
        self.logger.add_scalar('RDNet_RDLoss', rd_loss.item(), network_epoch*self.rd_epochs*len(train_loader) + epoch*len(train_loader) + batch_idx)
        if batch_idx % 200 == 0:
          print('Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tRD Total Loss: {:.6f}'.format(
            epoch, network_epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))       
        del base_output, rd_output, adv_output, data, target
  
  def train_adv(self, train_loader, network_epoch):
    for epoch in range(self.ad_epochs):
      self.train()
      for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(self.device).type(torch.float), target.to(self.device).type(torch.float)
        base_output = self.basenetwork(data)
        adv_output = self.adversary(base_output.detach())
        self.ad_optimizer.zero_grad()
        loss = self.ad_criterion(adv_output, target[:,1].unsqueeze(1))
        loss.backward()
        self.ad_optimizer.step()

        # Bookkeeping
        self.logger.add_scalar('AdvNet_AdvLoss', loss.item(), network_epoch*self.ad_epochs*len(train_loader) + epoch*len(train_loader) + batch_idx)
        if batch_idx % 200 == 0:
          print('Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tAdv Loss: {:.6f}'.format(
            epoch, network_epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
        del base_output, adv_output, data, target
        
  def train_network(self, train_loader, test_loader):
    self.pretrain_rd(train_loader)
    self.pretrain_ad(train_loader)
    for epoch in range(self.epochs):
      # self.train_adv(train_loader, epoch)
      self.train_rd(train_loader, epoch)
      self.test(epoch, test_loader)  

  def test(self, epoch, test_loader):
    self.eval()
    test_loss = 0
    test_adv_loss = 0
    test_rd_loss = 0
    correct = 0
    global global_epoch, global_batchid
    global_epoch = epoch 
    with torch.no_grad():
        for batch_id, tup in enumerate(test_loader):
            global_batchid = batch_id
            data, target = tup
            data, target = data.to(self.device).type(torch.float), target.to(self.device).type(torch.float)
            # plt.imshow(data[0].transpose(0,1).transpose(1,2).squeeze())
            # plt.show()
            base_output = self.basenetwork(data)
            rd_output = self.racedetector(base_output)
            adv_output = self.adversary(base_output)

            test_rd_loss += self.rd_criterion(rd_output, target[:,0].unsqueeze(1)).item()
            test_adv_loss += self.ad_criterion(adv_output, target[:,1].unsqueeze(1)).item()
            test_loss = test_rd_loss + test_adv_loss
            
            pred = (rd_output > 0.5) * 1.0
            correct += pred.eq(target[:,0].view_as(pred)).sum().item()
            if(batch_id == 0):
              global_logger.add_images("Input Image", vutils.make_grid(data.cpu(), padding=2).unsqueeze(0),  epoch)
    test_loss /= len(test_loader)
    self.logger.add_scalar('Test_Loss', test_loss, epoch)
    self.logger.add_scalar('Test_Adv_Loss', test_adv_loss, epoch)
    self.logger.add_scalar('Test_Rd_Loss', test_rd_loss, epoch)
    self.logger.add_scalar('Test_Accuracy', correct/len(test_loader.dataset), epoch)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Build Model

In [0]:
df = pd.read_csv('D:\\LabelFile_2000_updated.csv')
df.head()
trainData = df[df.train == False]
validData = df[df.train == True]

In [0]:
dataFolder = "D:\\"
train_dataset = CancerDataset(dataFolder, trainData) 
test_dataset = CancerDataset(dataFolder, validData) 

batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False )
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False )

In [17]:
bn = BaseNetwork('basic')
rn = RaceDetector(490)
an = Adversary(490)

adnw = AdvNetworkWrapper(bn, rn, an, bs=50, lr=0.001, rd_epochs = 1, ad_epochs= 5, epochs=10, alpha = 0.8)
adnw.train_network(train_loader, test_loader)

64
64
128
128
256
256
512
512
Number of Parameters:  22696882
Sun Dec 15 02:34:46 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 441.12       Driver Version: 441.12       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P1000       WDDM  | 00000000:01:00.0  On |                  N/A |
| 36%   47C    P0    N/A /  N/A |    827MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:              

KeyboardInterrupt: ignored

In [0]:
!nvidia-smi

# Train